In [22]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
EPOCH = 30
print(DEVICE)

cuda


In [23]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

BATCH_SIZE = 224
data_dir = '../dataset'
image_datasets = {x: ImageFolder(root = os.path.join(data_dir, x),
                                transform=data_transforms[x]) for x in ['train', 'valid']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'valid']} 
dataset_size = {x: len(image_datasets[x]) for x in ['train', 'valid']}

class_name = image_datasets['train'].classes

In [24]:
from torchvision import models

resnet = models.resnet50(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 450)
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

from torch.optim import lr_scheduler

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

C:\Users\kyc_o\anaconda3\envs\dl\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\kyc_o\anaconda3\envs\dl\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
ct = 0
for child in resnet.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad=False

In [29]:
import time
import copy

def train_resnet(model, criterion, optimizer, scheduler, num_epochs=30):   #num_epochs here!
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('-------------- epoch {} ----------------'.format(epoch + 1))
        since = time.time()
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                print('learning rate: ', l_r)
                
            epoch_loss = running_loss/dataset_size[phase]
            epoch_acc = running_corrects.double()/dataset_size[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
    print('Best Validation Accuracy: {:4f}'.format(best_acc))
    
    
    model.load_state_dict(best_model_wts)
    
    return model

In [30]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs = EPOCH)
torch.save(model_resnet50, 'resnet50.pt')

-------------- epoch 1 ----------------
learning rate:  [0.001]
train Loss: 3.5897 Acc: 0.2409
valid Loss: 2.0323 Acc: 0.4920
Completed in 1m 45s
-------------- epoch 2 ----------------
learning rate:  [0.001]
train Loss: 1.8286 Acc: 0.5411
valid Loss: 1.2527 Acc: 0.6716
Completed in 1m 5s
-------------- epoch 3 ----------------
learning rate:  [0.001]
train Loss: 1.3311 Acc: 0.6513
valid Loss: 0.9719 Acc: 0.7364
Completed in 1m 6s
-------------- epoch 4 ----------------
learning rate:  [0.001]
train Loss: 1.0825 Acc: 0.7123
valid Loss: 0.8608 Acc: 0.7778
Completed in 1m 6s
-------------- epoch 5 ----------------
learning rate:  [0.001]
train Loss: 0.9259 Acc: 0.7496
valid Loss: 0.7438 Acc: 0.7956
Completed in 1m 6s
-------------- epoch 6 ----------------
learning rate:  [0.001]
train Loss: 0.7881 Acc: 0.7831
valid Loss: 0.6406 Acc: 0.8298
Completed in 1m 6s
-------------- epoch 7 ----------------
learning rate:  [0.0001]
train Loss: 0.6900 Acc: 0.8076
valid Loss: 0.6083 Acc: 0.8391
Co